In [83]:
import os
import re
import httpx

import edgar as edgar
from edgar.core import set_identity
from edgar._companies import Company
from edgar._filings import get_by_accession_number
import pandas as pd
from bs4 import BeautifulSoup
from rich import print
from dotenv import load_dotenv, find_dotenv

%load_ext rich

load_dotenv(find_dotenv())

set_identity("lakshya@insy695.com")

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


[00:00:10] INFO     Identity of the Edgar REST client set to [lakshya@insy695.com]                      ]8;id=799127;file:///Users/yyy/Library/CloudStorage/OneDrive-Personal/MMA/INSY669/assignment/INSY-669-Group-Project/.venv/lib/python3.11/site-packages/edgar/core.py\core.py]8;;\:]8;id=25848;file:///Users/yyy/Library/CloudStorage/OneDrive-Personal/MMA/INSY669/assignment/INSY-669-Group-Project/.venv/lib/python3.11/site-packages/edgar/core.py#153\153]8;;\

In [68]:
BASE_URL = "https://api.sec-api.io/"
client = httpx.Client(
    base_url=BASE_URL,
    timeout=60.0
)


In [69]:
def parse_html_filing(filing):
    html_ = filing.html()
    regex = re.compile(r"(>Item(\s|&#160;|&nbsp;)(7A|7)\.{0,1})|(ITEM\s(7A|7)\.{0,1})")
    matches = regex.finditer(html_)

    match_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

    match_df.columns = ["item", "start", "end"]
    match_df["item"] = match_df.item.str.lower()

    match_df.replace("&#160;", " ", regex=True, inplace=True)
    match_df.replace("&nbsp;", " ", regex=True, inplace=True)
    match_df.replace(" ", " ", regex=True, inplace=True)
    match_df.replace("\.", "", regex=True, inplace=True)
    match_df.replace(">", "", regex=True, inplace=True)

    match_df = match_df.sort_values("start", ascending=True).drop_duplicates(
        subset=["item"], keep="last"
    )

    if len(match_df["start"].values) > 1:
        item_7_raw = html_[match_df["start"].values[0] : match_df["start"].values[1]]
    else:
        item_7_raw = html_[match_df["start"].values[0] :]

    item_7_content = BeautifulSoup(item_7_raw, "lxml").get_text("\n\n")

    return item_7_content

In [85]:
def extract_section(ticker, year):
    company = Company(ticker)
    filings_df = company.get_filings(form="10-K").to_pandas()
    filings_df["year"] = pd.to_datetime(filings_df["reportDate"]).dt.year
    accession_number = filings_df.query(f"year == {year}")["accession_number"].values[0]

    filing = get_by_accession_number(accession_number)

    item_7_content_html = parse_html_filing(filing)

    extractor_params = {
        "url": filing.url,
        "item": "7",
        "type": "text",
        "token": os.getenv("SEC_API_KEY"),
    }

    response = client.get("extractor", params=extractor_params)

    item_7_extracted = response.text

    # Save to files
    with open(f"extracted/{ticker}_{year}_item_7_extracted.txt", "w") as f:
        f.write(item_7_extracted)

    with open(f"html/{ticker}_{year}_item_7.txt", "w") as f:
        f.write(item_7_content_html)

    print(f"{ticker} {year} ITEM 7 saved")

    return filing


In [86]:
companies_code = pd.read_csv("./data/constituents.csv")

for company_code in companies_code['Symbol']:
    try:
        extract_section(company_code, 2020)
    except Exception as e:
        print(f"Extraction failed for company: {e}")
        continue

MMM 2020 ITEM 7 saved

AOS 2020 ITEM 7 saved

ABT 2020 ITEM 7 saved

ABBV 2020 ITEM 7 saved

Extraction failed for company: 'NoneType' object has no attribute 'get_filings'

...

Extraction failed for company: 'NoneType' object has no attribute 'get_filings'

Extraction failed for company: 'NoneType' object has no attribute 'get_filings'

Extraction failed for company: 'NoneType' object has no attribute 'get_filings'

DG 2020 ITEM 7 saved

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2                                                                                              │
│   3 for company_code in companies_code['Symbol']:                                                │
│   4 │   try:                                                                                     │
│ ❱ 5 │   │   extract_section(company_code, 2020)                                                  │
│   6 │   except Exception as e:                                                                   │
│   7 │   │   print(f"Extraction failed for company: {e}")                                         │
│   8 │   │   continue                                                                             │
│                                                                                                  │
│ in extract_section:2                                                                             │
│                                                                                                  │
│    1 def extract_section(ticker, year):                                                          │
│ ❱  2 │   company = Company(ticker)                                                               │
│    3 │   filings_df = company.get_filings(form="10-K").to_pandas()                               │
│    4 │   filings_df["year"] = pd.to_datetime(filings_df["reportDate"]).dt.year                   │
│    5 │   accession_number = filings_df.query(f"year == {year}")["accession_number"].values[0]    │
│                                                                                                  │
│ /Users/yyy/Library/CloudStorage/OneDrive-Personal/MMA/INSY669/assignment/INSY-669-Group-Project/ │
│ .venv/lib/python3.11/site-packages/edgar/_companies.py:641 in get_entity                         │
│                                                                                                  │
│   638 │   │   │   return r.json()                                                                │
│   639 │   │   r.raise_for_status()                                                               │
│   640                                                                                            │
│ ❱ 641                                                                                            │
│   642 @lru_cache(maxsize=32)                                                                     │
│   643 def get_entity_submissions(cik: int,                                                       │
│   644 │   │   │   │   │   │      include_old_filings: bool = True) -> EntityData:                │
│                                                                                                  │
│ /Users/yyy/Library/CloudStorage/OneDrive-Personal/MMA/INSY669/assignment/INSY-669-Group-Project/ │
│ .venv/lib/python3.11/site-packages/edgar/_companies.py:378 in for_ticker                         │
│                                                                                                  │
│   375 │   │   │   │   │   accession_number: Union[str, List] = None,                             │
│   376 │   │   │   │   │   file_number: Union[str, List] = None,                                  │
│   377 │   │   │   │   │   filing_date: Union[str, Tuple[str, str]] = None,                       │
│ ❱ 378 │   │   │   │   │   date: Union[str, Tuple[str, str]] = None,                              │
│   379 │   │   │   │   │   is_xbrl: bool = None,                                                  │
│   380 │   │   │   │   │   is_inline_xbrl: bool = None,                                           │
│   381 │   │   │   │   │   sort_by: Union[str, List[Tuple[st

In [59]:
extract_section("NFLX", 2020)

/Users/yyy/Library/CloudStorage/OneDrive-Personal/MMA/INSY669/assignment/INSY-669-Group-Project/.venv/lib/python3.1
1/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

NFLX 2020 ITEM 7 saved


╭──────────────────────────────────────── 10-K 📊 filing for NETFLIX INC ─────────────────────────────────────────╮
│                                                                                                                 │
│   Accession Number       Filing Date   Company       CIK                                                        │
│  ────────────────────────────────────────────────────────────                                                   │
│   0001065280-21-000040   2021-01-28    NETFLIX INC   1065280                                                    │
│                                                                                                                 │
│                                                                                                                 │
│   Links: 🏠 Homepage 📄 Primary Document 📜 Full Submission Text                                                │
│  ────────────────────────────────────────────────────────────────────────